## Quantities & Prices (Wright 1934)



Consider a data generating process for $(q,p)$ based
on Goldberger (1972), who in turn is describing the work of Sewall
Wright (1934).  The demand-supply system is
$$
   q_D = \alpha p + u\qquad q_S = \beta p + v\qquad q_D = q_S,
$$
where $(u,v)$ are unobserved shocks to demand and supply,
respectively. 



In [1]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid

# Structural parameters;
(α,β) = (-1,2)     
σ = {'u':1/2,'v':1/3}
μ = {'u':2,'v':-1}

# u,v assumed independent
u = iid.norm(loc=μ['u'], scale=σ['u'])  # Demand shocks
v = iid.norm(loc=μ['v'], scale=σ['v'])  # Supply shocks

# Reduced form coefficients
π = [[-β/(α - β), -1/(α - β)],
     [ α/(α - β), 1/(α - β)]]

# Generate N realizations of system
# Outcomes Y have columns (q,p)
N = 10

# Arrange shocks into an Nx2 matrix
U = np.c_[u.rvs(N), v.rvs(N)]

# Matrix product gives [q,p]; label by putting into df
df = pd.DataFrame(U@π,columns=['q','p'])
Udf = pd.DataFrame(U,columns=['u','v']) # For future reference

We can interrogate these data:



In [1]:
df

And compute the linear correlation&#x2026;



In [1]:
df.corr()

Or more generally the covariance matrix:



In [1]:
C=df.cov()
C

From which we can calculate the linear regression coefficient
of $p = a + bq + e$:



In [1]:
C.loc['p','q']/C.loc['q','q']

And learn about the probability density&#x2026;



In [1]:
from scipy import stats
import numpy as np
%matplotlib notebook

# Estimate joint density of (q,p)
pdf = stats.gaussian_kde(df.T).pdf 

ax = df.plot.scatter(x='q',y='p')

v = ax.axis()
Q = np.mgrid[v[0]:v[1]:100j].tolist()
P = np.mgrid[v[2]:v[3]:100j].tolist()

_ = ax.contour(Q,P,np.array([[pdf((q,p))[0] for p in P] for q in Q]))

#### Counterfactual Demand & Supply Schedules



What are the actual *counterfactual* demand and
supply schedules?  This is the kind of thing that Frisch described as
&ldquo;hypothetical experiments.&rdquo;  The schedules respond to shocks $u$ and $v$, respectively,
yielding



In [1]:
qmax = df['q'].max()
qmin = df['q'].min()

Q = pd.DataFrame({'min':np.maximum(0,df['q']-0.3*(qmax-qmin)),
                  'max':np.minimum(qmax*1.2,df['q']+0.3*(qmax-qmin)),
                  'miss':-1})

# Inverse counterfactual demand & supply (for plotting)
D = Q.add(-Udf['u'],axis=0)/α  
S = Q.add(-Udf['v'],axis=0)/β

counterfactual=pd.DataFrame({'S':S.stack(),
                             'D':D.stack(),
                             'Q':Q.stack()})

counterfactual=counterfactual.replace(-1,np.nan)

_ = counterfactual.plot(x='Q')

#### Controlling Price



Consider the question: what would expected demand be if we *fixed*
    the price at $p_0$?  Expected supply?



#### Average Causal Effect of a Change in Price



What would expected demand be if we *observed* that the price was $p_0$?



#### Price Change *Ceteris Paribus*



Suppose we *observe* prices and quantities $(p_0,q_0)$.  How *would*
    we expect the quantity demanded to change if prices were instead
    fixed at $p_1$, *ceteris paribus*?

